In [2]:
import torch
from torch import optim, nn
from torch.autograd import Variable
import syft as sy
hook = sy.TorchHook(torch)
# import pixiedust


Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/Users/adamhall/anaconda3/envs/pysyft/lib/python3.7/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.0.so'


In [3]:
@property
def location(self):
    m = self.__getitem__(0)
    w = m.weight[0]
    return w.location

nn.Sequential.location = location

In [4]:
# A Toy Dataset
x = torch.tensor([[0,0,0,0],[1,0,0,0],[0,1,0,0],[0,0,1,0],[1,1,0,0],[1,0,1,0],[0,1,1,0],[1,1,1,0],[0,0,0,1],[1,0,0,1],[0,1,0,1],[0,0,1,1],[1,1,0,1],[1,0,1,1],[0,1,1,1],[1,1,1,1.]])
x.requires_grad_()
target = torch.tensor([[0],[0],[0],[0],[0],[0],[0],[0],[1],[1],[1],[1],[1],[1],[1],[1.]])


#   Variables for performance metrics
epochs = 20
lr = 0.2
counter = 0

# Define 2 chained models
models = [
    nn.Sequential(
        nn.Linear(4, 3),
        nn.Tanh()
    ),
    nn.Sequential(
        nn.Linear(3, 1),
        nn.Sigmoid()
    )
]

# Create optimisers for each segment and link to their segment
optimizers = [
    optim.SGD(params=model.parameters(),lr=lr)
    for model in models
]

# create some workers
alice = sy.VirtualWorker(hook, id="alice")
bob = sy.VirtualWorker(hook, id="bob")
workers = alice, bob

# Send Model Segments and Data to starting locations
model_locations = [alice, bob]

for model, location in zip(models, model_locations):
    model.send(location)
    
x = x.send(models[0].location)
target = target.send(models[1].location)

In [7]:
# %%pixie_debugger

def train():
    # Training Logic
    for iter in range(epochs):

        # 1) erase previous gradients (if they exist)
        for opt in optimizers:
            opt.zero_grad()

        # 2) make a prediction
        a  = models[0](x)

        # 3) send the activation signal to the next model
        a_to_send = a.detach()
        remote_a = a_to_send.move(models[1].location)
        # re-enable autograd here
        remote_a.requires_grad_()

        pred =  models[1](remote_a)

        # 3) calculate how much we missed
        loss = ((pred - target)**2).sum()

        # 4) figure out which weights caused us to miss
        loss.backward()
        
        # 5) Backprop gradient to model behind

        grad_a = remote_a.grad
        
        
        # It looks like the .grad method of grabbing gradients remotely no longer works
        A_gradient = grad_a.location._objects[grad_a.id_at_location]
        A_tensor = remote_a.location._objects[remote_a.id_at_location]
        if A_gradient.sum() == A_tensor.sum():
            print("gradient returned by .grad is the same as the original tensor")
        
        # The gradient is still there though it just isn't accessible while remote.
        # Uncomment code below to verify this;
        # remote_a = remote_a.get()
        # print(remote_a)
        # print(remote_a.grad)
        

        grad_a.move(models[0].location)
                
        # 5) backpropagate gradient for thsi layer.
        a.backward(grad_a)

        # 5) change the weights
        for opt in optimizers:
            opt.step()

        # 6) print our progress
        # Do not use .data
        print(loss.get())
        
train()

gradient returned by .grad is the same as the original tensor
tensor(4.0154, requires_grad=True)
gradient returned by .grad is the same as the original tensor
tensor(3.9722, requires_grad=True)
gradient returned by .grad is the same as the original tensor
tensor(3.9993, requires_grad=True)
gradient returned by .grad is the same as the original tensor
tensor(4.1374, requires_grad=True)
gradient returned by .grad is the same as the original tensor
tensor(4.0259, requires_grad=True)
gradient returned by .grad is the same as the original tensor
tensor(3.9747, requires_grad=True)
gradient returned by .grad is the same as the original tensor
tensor(4.0483, requires_grad=True)
gradient returned by .grad is the same as the original tensor
tensor(3.9968, requires_grad=True)
gradient returned by .grad is the same as the original tensor
tensor(3.9997, requires_grad=True)
gradient returned by .grad is the same as the original tensor
tensor(3.9662, requires_grad=True)
gradient returned by .grad is 